# GPMS Functional Testing

## Imports and Functions

In [ ]:
import requests
import json
import datetime as dt

from Cryptodome.Cipher import AES
from Crypto.Signature import pkcs1_15 as PKC
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA256


In [ ]:
AES_BLOCK_SIZE = 16

def PKCS7_pad(data):
    padsize = AES_BLOCK_SIZE - len(data) % AES_BLOCK_SIZE
    return data + bytes([padsize]) * padsize

def PKCS7_unpad(data):
    offset = data[-1]
    return data[:-offset]

In [ ]:
def encrypt_aes(text: str, key: str) -> str:
    iv = '0' * AES.block_size * 2
    iv = bytes.fromhex(iv)

    text = text.encode()
    key = key.encode()

    cipher = AES.new(key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(PKCS7_pad(text))
    ciphertext = ciphertext.hex()
    return ciphertext

def decrypt_aes(text: str, key: str) -> str:
    iv = '0' * AES.block_size * 2
    iv = bytes.fromhex(iv)

    ciphertext = bytes.fromhex(text)
    key = key.encode()
    
    cipher = AES.new(key, AES.MODE_CBC, iv)
    text = cipher.decrypt(ciphertext)

    text = PKCS7_unpad(text).decode()

    return text

def verify(signature: str, message: str):
    f = open('../privkey.pem', 'rb')
    key = RSA.importKey(f.read())
    f.close()

    signature = bytes.fromhex(signature)
    hash_ = SHA256.new(message.encode())
    PKC.new(key).verify(hash_, signature)

In [ ]:
def encrypt_payload(header: dict, body: dict, api_key: str, shared_secret: str):
    encrytped_body = encrypt_aes(json.dumps(body), api_key)
    payload = {
        "header": json.dumps(header),
        "body": encrytped_body
    }
    encrytped_payload = encrypt_aes(json.dumps(payload), shared_secret)
    return encrytped_payload

def decrypt_payload(r: requests.Response, api_key: str, shared_secret: str, signature: str):
    if r.status_code in (200, 201):
        response = decrypt_aes((r.content).decode(), api_key)
    else:
        response = decrypt_aes((r.content).decode(), shared_secret)
    
    response_json = json.loads(response)
    print(json.dumps(response_json, indent=4))

    response_header = json.loads(response_json['header']) 
    verify(response_header['signature'], signature)

    response_body = json.loads(response_json['body']) 
    return response_body

## Inject Environment Variables

In [ ]:
from dotenv import load_dotenv
from pathlib import Path
import os

dotenv_path = Path('../app.env')
load_dotenv(dotenv_path=dotenv_path)

SIGNATURE_MSG = os.getenv('SIGNATURE_MSG')
SHARED_SECRET = os.getenv('SHARED_SECRET')
TEST_API_KEY = os.getenv('TEST_API_KEY')
TEST_SIGNATURE = os.getenv('TEST_SIGNATURE')

## A Simple Test

In [ ]:
text = 'The rooster crows at midnight!'
ciphertext = encrypt_aes(text, SHARED_SECRET)
result = decrypt_aes(ciphertext, SHARED_SECRET)
print(result)

In [ ]:
verify(TEST_SIGNATURE, SIGNATURE_MSG)

## Functional Testing

In [ ]:
domain = "https://753oz794d3.execute-api.ap-southeast-2.amazonaws.com"

### Password Generation

In [ ]:
# happy flow

url = f"{domain}/v1/password/generate"
header = {
    "clientId": 10000
}
body = {
    "clientId":10000,
    "entryId": 10000,
    "userInput":"gooooogle",
    "token": "xk034#9ucpx@k03-2in9xb2"
}

encrytped_payload = encrypt_payload(header, body, TEST_API_KEY, SHARED_SECRET)
r = requests.post(url, json=encrytped_payload)
print(f"status code: {r.status_code}")

response_body = decrypt_payload(r, TEST_API_KEY, SHARED_SECRET, SIGNATURE_MSG)

In [ ]:
# wrong field type

url = f"{domain}/v1/password/generate"
header = {
    "clientId": 10000
}
body = {
    "clientId": 10000,
    "entryId": "10001", # wrong type
    "userInput":"google",
    "token": "xk034#9ucpx@k03-2in9xb2"
}

encrytped_payload = encrypt_payload(header, body, TEST_API_KEY, SHARED_SECRET)
r = requests.post(url, json=encrytped_payload)
print(f"status code: {r.status_code}")

response_body = decrypt_payload(r, TEST_API_KEY, SHARED_SECRET, SIGNATURE_MSG)

In [ ]:
# missing fields in request

url = f"{domain}/v1/password/generate"
header = {
    "clientId": 10000
}
body = {
    "clientId":10000,
    "entryId": 10001,
    "userInput":"google"
    # missing field
}

encrytped_payload = encrypt_payload(header, body, TEST_API_KEY, SHARED_SECRET)
r = requests.post(url, json=encrytped_payload)
print(f"status code: {r.status_code}")

response_body = decrypt_payload(r, TEST_API_KEY, SHARED_SECRET, SIGNATURE_MSG)

In [ ]:
# password entry not found in database 

url = f"{domain}/v1/password/generate"
header = {
    "clientId": 10000
}
body = {
    "clientId":10000,
    "entryId": 20000, # invalid entry id
    "userInput":"google",
    "token": "xk034#9ucpx@k03-2in9xb2"
}

encrytped_payload = encrypt_payload(header, body, TEST_API_KEY, SHARED_SECRET)
r = requests.post(url, json=encrytped_payload)
print(f"status code: {r.status_code}")

response_body = decrypt_payload(r, TEST_API_KEY, SHARED_SECRET, SIGNATURE_MSG)

In [ ]:
# wrongly encrypted request

url = f"{domain}/v1/password/generate"
header = {
    "clientId": 10000
}
body = {
    "clientId":10000,
    "entryId": 20000,
    "userInput":"google",
    "token": "xk034#9ucpx@k03-2in9xb2"
}

encrytped_payload = encrypt_payload(header, body, SHARED_SECRET, TEST_API_KEY)
r = requests.post(url, json=encrytped_payload)
print(f"status code: {r.status_code}")

response_body = decrypt_payload(r, TEST_API_KEY, SHARED_SECRET, SIGNATURE_MSG)

### Entry Creation

In [ ]:
# happy flow

url = f"{domain}/v1/entry/create"
header = {
    "clientId": 10000
}
metadata = {
    "url": "www.google.com",
    "createdAt": dt.datetime.now(tz = dt.timezone.utc).strftime("%Y-%m-%dT%H:%M:%S%z"),
    "ipAddress": "8.8.8.8",
    "type": "search engine"
}
body = {
    "clientId":10000,
    "entryReferenceId":"G0001",
    "passwordLength":15,
    "siteName":"Google",
    "siteType":"Website",
    "metadata": json.dumps(metadata),
    "username":"abc"
}

encrytped_payload = encrypt_payload(header, body, TEST_API_KEY, SHARED_SECRET)
r = requests.post(url, json=encrytped_payload)
print(f"status code: {r.status_code}")

response_body = decrypt_payload(r, TEST_API_KEY, SHARED_SECRET, SIGNATURE_MSG)

In [ ]:
# wrong field type

url = f"{domain}/v1/entry/create"
header = {
    "clientId": 10000
}
body = {
    "clientId":10000,
    "entryReferenceId":"G0001",
    "passwordLength":"15",
    "siteName":"Google",
    "siteType":"Website",
    "metadata":"www.google.com",
    "username":"abc"
}

encrytped_payload = encrypt_payload(header, body, TEST_API_KEY, SHARED_SECRET)
r = requests.post(url, json=encrytped_payload)
print(f"status code: {r.status_code}")

response_body = decrypt_payload(r, TEST_API_KEY, SHARED_SECRET, SIGNATURE_MSG)

In [ ]:
# missing fields

url = f"{domain}/v1/entry/create"
header = {
    "clientId": 10000
}
body = {
    "clientId":10000,
    "passwordLength":15,
    "siteName":"Google",
    "siteType":"Website",
    "metadata":"www.google.com",
    "username":"abc"
}

encrytped_payload = encrypt_payload(header, body, TEST_API_KEY, SHARED_SECRET)
r = requests.post(url, json=encrytped_payload)
print(f"status code: {r.status_code}")

response_body = decrypt_payload(r, TEST_API_KEY, SHARED_SECRET, SIGNATURE_MSG)

In [ ]:
# Less than minimum password length

url = f"{domain}/v1/entry/create"
header = {
    "clientId": 10000
}
body = {
    "clientId":10000,
    "entryReferenceId":"G0001",
    "passwordLength":4,
    "siteName":"Google",
    "siteType":"Website",
    "metadata":"www.google.com",
    "username":"abc"
}

encrytped_payload = encrypt_payload(header, body, TEST_API_KEY, SHARED_SECRET)
r = requests.post(url, json=encrytped_payload)
print(f"status code: {r.status_code}")

response_body = decrypt_payload(r, TEST_API_KEY, SHARED_SECRET, SIGNATURE_MSG)

In [ ]:
# client ID not found in database

url = f"{domain}/v1/entry/create"
header = {
    "clientId": 10000
}
body = {
    "clientId":20000,
    "entryReferenceId":"G0001",
    "passwordLength":15,
    "siteName":"Google",
    "siteType":"Website",
    "metadata":"www.google.com",
    "username":"abc"
}

encrytped_payload = encrypt_payload(header, body, TEST_API_KEY, SHARED_SECRET)
r = requests.post(url, json=encrytped_payload)
print(f"status code: {r.status_code}")

response_body = decrypt_payload(r, TEST_API_KEY, SHARED_SECRET, SIGNATURE_MSG)

In [ ]:
# wrongly encrypted request

url = f"{domain}/v1/entry/create"
header = {
    "clientId": 10000
}
body = {
    "clientId":10000,
    "entryReferenceId":"G0001",
    "passwordLength":15,
    "siteName":"Google",
    "siteType":"Website",
    "metadata":"www.google.com",
    "username":"abc"
}

encrytped_payload = encrypt_payload(header, body, SHARED_SECRET, TEST_API_KEY)
r = requests.post(url, json=encrytped_payload)
print(f"status code: {r.status_code}")

response_body = decrypt_payload(r, TEST_API_KEY, SHARED_SECRET, SIGNATURE_MSG)